In [11]:
import json
import urllib
import pandas as pd
from datetime import datetime
def get_schedule(year):
    url = "http://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard?dates={}".format(year)
    resp = urllib.request.urlopen(url)
    txt = json.loads(resp.read())['leagues'][0]['calendar']
#     print(len(txt))
    reg = pd.DataFrame(txt[0]['entries'])
    post = pd.DataFrame(txt[1]['entries'])
#     offseason = pd.DataFrame(txt[2]['entries']) if (len(txt) > 2) else pd.DataFrame()
    full_schedule = pd.concat([reg,post], ignore_index=True)
    full_schedule['year']=year
    full_schedule['daysForward'] = (pd.to_datetime(full_schedule['endDate'])-pd.to_datetime(full_schedule['startDate'])).dt.days+1
    date = pd.to_datetime(pd.to_datetime(full_schedule['startDate']).dt.date,format='%Y-%m-%d')
    full_schedule['str_startDate']=date.apply(lambda x: x.strftime('%Y%m%d'))
    full_schedule['url']="http://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard?dates="
    full_schedule['url'] = full_schedule[['url','str_startDate']].apply(lambda x: 
                        ''.join(x),axis=1)
    full_schedule['daysForward'] = full_schedule['daysForward'].apply(str)
    full_schedule['url'] = full_schedule[['url','daysForward']].apply(lambda x: 
                        '&daysForward='.join(x),axis=1)
    return (full_schedule)

full_schedule = get_schedule(2018)
pd.set_option('display.max_colwidth',None)
full_schedule.to_json('2018_sched.json', orient='records')


In [12]:
for x in range(2002,2021):
    full_schedule = get_schedule(x)
    full_schedule.to_json(f"./frontend/cfb/schedules/{str(x)}.json",orient='records')